In [ ]:
%%capture
import os
from pathlib import Path

from dj_notebook import activate

env_file = os.environ["META_ENV"]
reports_folder = Path(os.environ["META_REPORTS_FOLDER"])
analysis_folder = Path(os.environ["META_ANALYSIS_FOLDER"])
pharmacy_folder = Path(os.environ["META_PHARMACY_FOLDER"])
plus = activate(dotenv_file=env_file)

In [ ]:
import numpy as np
import pandas as pd

from datetime import date
from django_pandas.io import read_frame

from edc_model_to_dataframe.read_frame_edc import read_frame_edc
from edc_registration.models import RegisteredSubject

from meta_prn.models import EndOfStudy, OffSchedule
from meta_subject.models import FollowupVitals

In [ ]:
df_rs = read_frame(RegisteredSubject.objects.values("subject_identifier", "gender", "dob").all())
df_rs["dob"] = pd.to_datetime(df_rs["dob"])
today = pd.to_datetime(date.today())
age = today.year - df_rs['dob'].dt.year
df_rs['age_in_years'] = age

df_vitals = read_frame_edc(FollowupVitals.objects.all())
df_eos = read_frame_edc(EndOfStudy.objects.all())
df_offschedule = read_frame_edc(OffSchedule.objects.all())

In [ ]:
# merge vitals, registered subject
df = pd.merge(df_rs, df_vitals, on="subject_identifier")

# select the last record for each subject
df['report_datetime'] = pd.to_datetime(df['report_datetime'])
df = df.sort_values(by=['subject_identifier', 'report_datetime'])
df = df.drop_duplicates(subset='subject_identifier', keep='last')

# drop any subjects off study
df = df[~df['subject_identifier'].isin(df_eos['subject_identifier'].unique())]

# drop any subjects off-schedule (but still on study)
df = df[~df['subject_identifier'].isin(df_offschedule['subject_identifier'].unique())]

# cut/bin weight
df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
bins = [-np.inf, 35, 50, np.inf]
labels = ['<35', '35-49', '>=50']
df['weight_bin'] = pd.cut(df['weight'], bins=bins, labels=labels, right=False)

# df[['subject_identifier', 'visit_code', 'report_datetime', 'site_id', 'gender', 'weight', 'weight_bin']]

In [ ]:
# sample 20% subjects
stratified_sample = df.groupby(
    ['site_id', 'gender', 'weight_bin', 'age_in_years'],
    group_keys=False,
    observed=True
).sample(
    frac=0.20,
    random_state=981
)

In [ ]:
# randomize the stratified sample by site
randomized_df = stratified_sample.groupby(
    'site_id',
    group_keys=False,
    observed=True
).sample(n=20, random_state=876).reset_index()

In [ ]:
# add sid column, ...
randomized_df["sid"] = randomized_df.index+42000
randomized_df["date_generated"] = pd.Timestamp.now()
randomized_df = randomized_df.rename(columns={"visit_code": "last_visit_code", "report_datetime": "last_appt_datetime"})


In [ ]:
# review
randomized_df[['sid', 'subject_identifier', 'last_visit_code', 'last_appt_datetime', 'site_id', 'gender', 'age_in_years', 'weight_bin', 'date_generated']]

In [ ]:
# review
randomized_df.site_id.value_counts(normalize=True)

In [ ]:
# review
randomized_df.site_id.value_counts()

In [ ]:
# review
read_frame_edc(RegisteredSubject.objects.values("subject_identifier", "site_id").all()).site_id.value_counts(normalize=True)

In [ ]:
# export
randomized_df[['sid', 'subject_identifier', 'last_visit_code', 'last_appt_datetime', 'site_id', 'gender', 'age_in_years', 'weight_bin', 'date_generated']].to_csv(analysis_folder / "nanda_list_frac.csv")

In [44]:
# method 2
stratified_sample = df.groupby(
    ['site_id', 'gender', 'weight_bin', 'age_in_years'],
    group_keys=False,
    observed=True
).sample(
    n=1,
    random_state=981
)
# randomize the stratified sample by site
randomized_df = stratified_sample.groupby(
    'site_id',
    group_keys=False,
    observed=True
).sample(n=50, random_state=876).reset_index()
# add sid column, ...
randomized_df["sid"] = randomized_df.index + 42000
randomized_df["date_generated"] = pd.Timestamp.now()
randomized_df = randomized_df.rename(columns={"visit_code": "last_visit_code",
                                              "report_datetime": "last_appt_datetime"})


In [45]:
randomized_df[['sid', 'subject_identifier', 'last_visit_code', 'last_appt_datetime', 'site_id', 'gender', 'age_in_years', 'weight_bin', 'date_generated']].to_csv(analysis_folder / "nanda_list_n.csv")


In [46]:
randomized_df.site_id.value_counts(normalize=True)


site_id
10    0.2
20    0.2
30    0.2
40    0.2
60    0.2
Name: proportion, dtype: float64

In [47]:
randomized_df.site_id.value_counts()


site_id
10    50
20    50
30    50
40    50
60    50
Name: count, dtype: int64

In [48]:
randomized_df.subject_identifier.nunique()

250

In [49]:
len(randomized_df)

250